In [ ]:
#include <algoviz/SVG.hpp>
#include "Level.hpp"
#include "Protagonist.hpp"
#include "Item.hpp"

AlgoViz::clear();

SVG view = SVG(700, 500, "Level 1");
view.drawRect(0, 0, 700, 550);

view.setStatusMsg("Drücken Sie x zu beenden!");

Text anzeigeL = Text("Leben: 3", 15, 20, &view);
Text anzeigeP = Text("Punkte: 0", 15, 37, &view);


MouseState maus;

//Spieler
Protagonist mario = Protagonist(&view);

Hindernis hindernis = Hindernis();

int v;

//für die Items
list<Item*> items;
Item *item;

int muenzeAnzahl;
int feindAnzahl;
int marioBlockAnzahl;

//Karte des Levels
Level levelMap = Level(v, muenzeAnzahl, feindAnzahl, marioBlockAnzahl, &feld);

int blockadedauer = 0;
bool blockade = false; // Einmal gedrückt, ist der Sprung vorerst nicht beeinflussbar...man kann aber "Supersprung" aktivieren
bool blockade2 = false; // Blockiert jegliche Aktionen während des Runterspringens
int arrowUp = 0; // Speichert die Anzahl der "arrowUps", die während der Blockade gedrückt werden...
bool supersprungB = false;
string taste = "";
int zaehler1 = 0;
int zaehler2 = 0;

mario.setHindernis(&hindernis);

In [ ]:
//Karte füllen
levelMap.fuellMap();

//Items platzieren
levelMap.plaziereMuenze(muenzeAnzahl);
levelMap.plaziereFeind(feindAnzahl);
levelMap.plaziereMarioBlock(marioBlockAnzahl);

int i = 0;

while (taste != "x") {
    taste = view.lastKey();

    //Wenn die Karte zu ende es, wird sie von vorne wiederbefüllt
    if (i == 50) {
        
        for (int j = 0; j < 50; j++) {
            
            levelMap.move();
        }
        
        levelMap.fuellMap();
        int neuV = levelMap.getV() + 5;
        levelMap.setV(neuV);
        i = 0;
    }
    
    //für die drei Unterschiedliche Höhe
    for(int j = 0; j < 3; j++){
        
        if (levelMap.ist1(i,j)) { 
            
            levelMap.add(j);

        //Items-> boolfunktionen für Items fehlen in klasse level noch
        //stein
        }else if (levelMap.iststein(i,j)) {
            
            // Y-Position der Items festlegen
            int y = 500 - 25 - j * rand() % 150;
            item = new Item(700, y, 50, 50, 0, &feld);
            items.push_back(item);
            
        //muenze
        }else if (levelMap.istmuenze(i,j)) {
            
            // Y-Position der Items festlegen
            int y = 500 - 15 - j * rand() % 150;           
            item = new Item(700, y, 30, 30, 1, &feld);
            items.push_back(item);
            
        //feind
        }else if (levelMap.istfeind(i,j)) {
            
            item = new Item(700, 50, 60, 100, 2, &feld);
            items.push_back(item);
        }

    }
        
    //Items und Hindernssisse werden bewegt
    item->moveItems(items,v);
    levelMap.move();
    

    if ( taste == "ArrowUp" && blockade == false ) {
        if ( mario.getBildwert() == 2 ) {
            mario.switchBildaktiv("ArrowUp");
        } else if ( supersprungB == false && mario.getY() + 140 >= mario.getHindernisY() - 40 ) { 
            mario.springenhoch();
            blockade = true; 
        }
    } else if ( taste == "ArrowUp" && blockade == true ) {
        if ( mario.getY() + 140 >= mario.getHindernisY() - 40 && supersprungB == false && blockade2 == false ) {
            mario.springenhoch();
            arrowUp++;
            if ( mario.getY() + 140 < mario.getHindernisY() - 40 && arrowUp >= 4 ) { 
                arrowUp = 0;
                supersprungB = true;
                mario.supersprung();
            } else if ( mario.getY() + 140 < mario.getHindernisY() - 40 && arrowUp < 4 ) {
                arrowUp = 0;
                blockade2 = true;
            } 
        } else if ( mario.getY() + 280 >= mario.getHindernisY() - 40 && supersprungB == true && blockade2 == false ) {
            mario.springenhoch();
            if ( mario.getY() + 280 < mario.getHindernisY() - 40 ) {
                blockade2 = true;
            }
        } else if ( mario.getY() < mario.getHindernisY() - 40 && blockade2 == true ) {
            mario.springenrunter();
            if ( mario.getY() >= mario.getHindernisY() - 40 ) {
                blockade2 = false;
                blockade = false;
                supersprungB = false;
                mario.setSupersprung(false);
            }
        }
    } else if ( taste != "ArrowUp" && blockade == true ) {
        if ( mario.getY() + 140 >= mario.getHindernisY() - 40 && supersprungB == false && blockade2 == false ) {
            mario.springenhoch();
            if ( mario.getY() + 140 < mario.getHindernisY() - 40 && arrowUp >= 4 ) {
                arrowUp = 0;
                supersprungB = true;
                mario.supersprung();
            } else if ( mario.getY() + 140 < mario.getHindernisY() - 40 && arrowUp < 4 ) {
                arrowUp = 0;
                blockade2 = true;
            }
        } else if ( mario.getY() + 280 >= mario.getHindernisY() - 40 && supersprungB == true && blockade2 == false ) {
            mario.springenhoch();
            if ( mario.getY() + 280 < mario.getHindernisY() - 40 ) {
                blockade2 = true;
            }
        } else if ( mario.getY() < mario.getHindernisY() - 40 && blockade2 == true ) {
            mario.springenrunter();
            if ( mario.getY() >= mario.getHindernisY() - 40 ) {
                blockade2 = false;
                blockade = false;
                supersprungB = false;
                mario.setSupersprung(false);
            }
        }
    } else if ( taste == "ArrowDown" && mario.getY() >= mario.getHindernisY() - 40 ) {
        mario.switchBildaktiv("ArrowDown");
    }


    AlgoViz::sleep(5);
    i++;
    if (i == 150) {
        mario.switchBildpassiv();
        zaehler1 = 0;
    }
}

view.setStatusMsg("Ende!");